In [7]:
import numpy as np

In [8]:
def TT_to_tensor(factors): # error if dimension > 26
    if (len(factors) > 26):
        raise ValueError('error if dimension > 26')
    mult_str = ""
    counter = 0
    for _ in range(len(factors) - 1):
        mult_str = mult_str + chr(97 + counter) + chr(65 + counter) + "," + chr(65 + counter)
        counter += 1
    
    mult_str += chr(97 + counter)

    T = np.einsum(mult_str, *factors)
    return T

In [9]:
def F_norm(tensor):
    return np.sum(tensor ** 2)

def truncatedSVD(A, delta):
    u, s, v = np.linalg.svd(A, full_matrices=False)
    rank = (np.abs(s) >= delta).sum()
    return u[:, :rank], s[:rank], v[:rank, :]

def numel(tensor):
    return tensor.size

In [10]:
def TT_SVD(tensor, eps):
    d = len(tensor.shape)
    delta = eps / np.sqrt(d - 1) * F_norm(tensor)
    Ctens = tensor.copy()
    
    rk = 1
    r_prev = 1

    G_list = []
    for k in range(1, d):
        Ctens = Ctens.reshape(
            r_prev * tensor.shape[k - 1], 
            numel(Ctens) // (r_prev * tensor.shape[k - 1])
        )
        u, s, v = truncatedSVD(Ctens, delta)
        rk = len(s)
        if k > 1:
            Gk = u.reshape(r_prev, tensor.shape[k - 1], rk)
        else:
            Gk = u.reshape(tensor.shape[k - 1], rk)
        G_list.append(Gk)
        r_prev = rk
        Ctens =  np.diag(s) @ v
    G_list.append(Ctens)
    #G_list.append(Ctens.reshape(r_prev, Ctens.shape[1] // tensor.shape[-1], tensor.shape[-1]))
    return G_list
    

In [11]:
def TT_sum(G1, G2):
    G3 = []
    G3.append(np.hstack((G1[0], G2[0])))

    for i in range(1, len(G1) - 1):
        G_next = np.zeros([G1[i].shape[0] + G2[i].shape[0], G1[i].shape[1], G1[i].shape[2] + G2[i].shape[2]])
        G_next[:G1[i].shape[0], :, :G1[i].shape[2]] = G1[i]
        G_next[G1[i].shape[0]:, :, G1[i].shape[2]:] = G2[i]
        G3.append(G_next)

    G3.append(np.vstack((G1[-1], G2[-1])))
    return G3

In [12]:
def TT_squeeze(G, eps=1e-15):
    #QR ортогонализация

    Q, R = np.linalg.qr(G[0])
    G[0] = Q
    for i in range(1, len(G) - 1):
        G[i] = np.einsum("mk,krt->mrt", R, G[i])
        Q, R = np.linalg.qr(G[i].reshape(-1, G[i].shape[2]))
        G[i] = Q.reshape(G[i].shape[0], G[i].shape[1], -1)

    G[-1] = np.einsum("mk,kr->mr", R, G[-1])

    # SVD дожатиe
    u, s, G[-1] = np.linalg.svd(G[-1], full_matrices=False)
    singular_count = np.argwhere(np.cumsum(s[::-1] ** 2) / s[0] > eps)[0][0]
    rank = s.size - singular_count
    G[-1] = G[-1][:rank, :]
    U = u[:, :rank] * s[np.newaxis, :rank]

    for i in range(len(G) - 2, 0, -1):
        G[i] = np.einsum("mrt,tk->mrk", G[i], U)
        n, right_rank = G[i].shape[1], G[i].shape[2]
        G[i] = np.reshape(G[i], (-1, n * rank))
        u, s, G[i] = np.linalg.svd(G[i])
        singular_count = np.argwhere(np.cumsum(s[::-1] ** 2) / s[0] > eps)[0][0]
        rank = s.size - singular_count
        G[i] = G[i][:rank, :]
        G[i] = np.reshape(G[i], (-1, n, right_rank))
        U = u[:, :rank] * s[np.newaxis, :rank]

    G[0] = G[0] @ U
    return G

Дожатие при сложении тензора с самим собой

In [13]:
dimensions = [4, 5, 3, 7]
ranks = [2, 3, 2]

In [14]:
S1 = []
S1.append(np.random.normal(size = (dimensions[0], ranks[0])))

if (len(ranks) > 1):
    for i in range(1, len(ranks)):
        S1.append(np.random.normal(size = (ranks[i - 1], dimensions[i], ranks[i])))
S1.append(np.random.normal(size = (ranks[-1], dimensions[-1])))

In [15]:
S2 = TT_sum(S1, S1)
print ("TT ranks after sum: (", end = " ")
for i in range(len(S2) - 1):
    print(S2[i].shape[-1], end = " ")
print(")")

T1 = TT_to_tensor(S1)

TT_optimal_sum = TT_squeeze(S2)

print ("TT ranks after TT-squeez: (", end = " ")
for i in range(len(TT_optimal_sum) - 1):
    print(TT_optimal_sum[i].shape[-1], end = " ")
print(")")


T2 = TT_to_tensor(TT_optimal_sum)

print("relative error in Frobenius: ", np.linalg.norm(2 * T1 - T2) / np.linalg.norm(2 * T1))

TT ranks after sum: ( 4 6 4 )
TT ranks after TT-squeez: ( 2 3 2 )
relative error in Frobenius:  6.840172394562053e-16


Дожатие с матрицей Гильберта:    
$\frac{1}{i + j + k + l + s + 1}$

In [16]:
def f(i, j, k, l, s):
    return 1 / (i + j + k + l + s + 1)

In [17]:
n = 16
T1 = np.fromfunction(f, (n, n, n, n, n))

In [18]:
TT_1 = TT_SVD(T1, eps=1e-4)

print ("TT ranks after TT-SVD eps= 1e-4: (", end = " ")
for i in range(len(TT_1) - 1):
    print(TT_1[i].shape[-1], end = " ")
print(")")

TT_2 = TT_SVD(T1, eps=1e-10)

print ("TT ranks after TT-SVD eps= 1e-10: (", end = " ")
for i in range(len(TT_2) - 1):
    print(TT_2[i].shape[-1], end = " ")
print(")")

TT ranks after TT-SVD eps= 1e-4: ( 4 4 4 4 )
TT ranks after TT-SVD eps= 1e-10: ( 9 10 10 9 )


In [20]:
TT_sum_bad_ranks = TT_sum(TT_1, TT_2)
print ("TT ranks after sum: (", end = " ")
for i in range(len(TT_sum_bad_ranks) - 1):
    print(TT_sum_bad_ranks[i].shape[-1], end = " ")
print(")")


TT_optimal_sum = TT_squeeze(TT_sum_bad_ranks, eps = 1e-9)

print ("TT ranks after TT-squeez: (", end = " ")
for i in range(len(TT_optimal_sum) - 1):
    print(TT_optimal_sum[i].shape[-1], end = " ")
print(")")


TT ranks after sum: ( 13 14 14 13 )
TT ranks after TT-squeez: ( 6 7 7 6 )
